In [21]:
import pandas as pd
import networkx as nx

In [2]:
# Lecture du fichier des tables mergées avec les estimations m2 déjà enregistrées pour France Métropolitaine
df = pd.read_csv('./Tables_traitees/Table_avec_prixM2_estime_FranceMétropolitaine.csv',
                 sep=';',dtype={'3_Code département':'category', '3_Code INSEE (commune ou arrondissement)':str, 
                                '3_Code EPCI':str, '1_Code INSEE commune':str, '1_Num PDL':str,
                                '1_longitude':str, '1_code_commune_n':str, '2_Code INSEE':str,
                                '2_code_commune_n':str, '4_Code Postal':str, "5_INSEE_DEP":str, "5_ID":str,
                                "5_INSEE_COM":str,"5_code_commune_n":str,"6_insee":str,"4_Code INSEE":str,
                                "4_Code Département": 'category' ,'4_Statut':'category','4_Code Région':'category',
                               '3_Code Région':'category'})

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,3_Code région,3_Nom de la région,3_Code département,3_Code commune,3_Nom de la commune,3_Population municipale,3_Population comptée à part,3_Population totale,...,6_type,A_Prix_consolidé_vois_mean,A_Prix_estimé_vois_mean,B_Prix_consolidé_vois_median,B_Prix_estimé_vois_median,C_Prix_estime_mean,C1_Prix_estime_median,F_Prix_estimé_RFReg,G_Prix_estimé_LassoCV,F_Prix_estimé_RFReg2
0,0,0,84.0,Auvergne-Rhône-Alpes,01,1.0,L' Abergement-Clémenciat,767.0,18.0,785.0,...,NaN,1938.0,2167.500000,1938.0,2080.5,1631.0,1448.0,1590.398336,1997.681268,NaN
1,1,1,84.0,Auvergne-Rhône-Alpes,01,2.0,L' Abergement-de-Varey,243.0,1.0,244.0,...,NaN,1130.0,1503.879310,1130.0,1458.5,1631.0,1448.0,1398.761818,1938.198659,NaN
2,2,2,84.0,Auvergne-Rhône-Alpes,01,4.0,Ambérieu-en-Bugey,14081.0,437.0,14518.0,...,NaN,2005.0,1725.714286,2005.0,1678.0,1631.0,1448.0,2341.409735,2108.273069,NaN
3,3,3,84.0,Auvergne-Rhône-Alpes,01,5.0,Ambérieux-en-Dombes,1671.0,34.0,1705.0,...,NaN,2442.0,2647.857143,2442.0,2701.0,1631.0,1448.0,1954.337259,2002.436347,NaN
4,4,4,84.0,Auvergne-Rhône-Alpes,01,6.0,Ambléon,110.0,6.0,116.0,...,NaN,1155.0,1460.333333,1155.0,1287.0,1631.0,1448.0,1362.887999,1917.910983,NaN


In [5]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', '3_Code région', '3_Nom de la région',
       '3_Code département', '3_Code commune', '3_Nom de la commune',
       '3_Population municipale', '3_Population comptée à part',
       '3_Population totale', '3_Année recensement', '3_Année utilisation',
       '3_Code INSEE (commune ou arrondissement)',
       '3_Superficie de la commune', '3_Statut', '3_Code EPCI', '3_EPCI',
       '3_Densité_hab_par_km2', '5_ID', '5_INSEE_COM', '5_INSEE_DEP',
       '5_INSEE_REG', '5_CODE_EPCI', '5_NOM_COM_M', '5_POPULATION',
       '5_Nb_Ventes', '5_PrixMoyen_M2', '5_nb_digits_code_INSEE',
       '5_code_commune_n', '4_Code INSEE', '4_Code Postal', '4_Commune',
       '4_Département', '4_Région', '4_Statut', '4_Altitude Moyenne',
       '4_Superficie', '4_Population', '4_geo_point_2d', '4_geo_shape',
       '4_ID Geofla', '4_Code Commune', '4_Code Canton',
       '4_Code Arrondissement', '4_Code Département', '4_Code Région', '4_lat',
       '4_lon', '4_Densité_hab_p

In [7]:
col = ['6_insee', '6_nom', '6_nb_voisins',
       '6_insee_voisins', '6_noms_voisins', '6_cap_voisins', '6_type']

In [8]:
df[col].head()

,6_insee,6_nom,6_nb_voisins,6_insee_voisins,6_noms_voisins,6_cap_voisins,6_type
0,01001,L'Abergement-Clémenciat,6.0,01412|01093|01028|01146|01351|01188,Sulignat|Châtillon-sur-Chalaronne|Baneins|Domp...,48|144|203|234|260|322,NaN
1,01002,L'Abergement-de-Varey,6.0,01056|01277|01384|01007|01363|01199,Boyeux-Saint-Jérôme|Nivollet-Montgriffon|Saint...,43|115|172|265|302|344,NaN
2,01004,Ambérieu-en-Bugey,7.0,01384|01421|01041|01345|01089|01007|01149,Saint-Rambert-en-Bugey|Torcieu|Bettant|Saint-D...,102|151|190|250|284|347|360,NaN
3,01005,Ambérieux-en-Dombes,7.0,01382|01207|01261|01362|01318|01398|01446,Sainte-Olive|Lapeyrouse|Monthieux|Saint-Jean-d...,43|102|145|193|230|272|310,NaN
4,01006,Ambléon,6.0,01358|01110|01117|01216|01233|01190,Saint-Germain-les-Paroisses|Colomieu|Conzieu|L...,37|120|164|263|308|341,NaN


In [40]:
dict_com = {x['6_insee']:x['6_nom'] for i,x in df.iterrows()}
dict_com

{'01001': "L'Abergement-Clémenciat",
 '01002': "L'Abergement-de-Varey",
 '01004': 'Ambérieu-en-Bugey',
 '01005': 'Ambérieux-en-Dombes',
 '01006': 'Ambléon',
 '01007': 'Ambronay',
 '01008': 'Ambutrix',
 '01009': 'Andert-et-Condon',
 '01010': 'Anglefort',
 '01011': 'Apremont',
 '01012': 'Aranc',
 '01013': 'Arandas',
 '01014': 'Arbent',
 '01015': 'Arboys-en-Bugey',
 '01016': 'Arbigny',
 '01017': 'Argis',
 '01019': 'Armix',
 '01021': 'Ars-sur-Formans',
 '01022': 'Artemare',
 '01023': 'Asnières-sur-Saône',
 '01024': 'Attignat',
 '01025': 'Bâgé-Dommartin',
 '01026': 'Bâgé-le-Châtel',
 '01027': 'Balan',
 '01028': 'Baneins',
 '01029': 'Beaupont',
 '01030': 'Beauregard',
 '01031': 'Bellignat',
 '01032': 'Béligneux',
 '01033': 'Valserhône',
 '01034': 'Belley',
 '01035': 'Belleydoux',
 '01036': 'Valromey-sur-Séran',
 '01037': 'Bénonces',
 '01038': 'Bény',
 '01039': 'Béon',
 '01040': 'Béréziat',
 '01041': 'Bettant',
 '01042': 'Bey',
 '01043': 'Beynost',
 '01044': 'Billiat',
 '01045': 'Birieux',
 '

In [47]:
df_com = pd.DataFrame([(key, value) for key, value in dict_com.items()], columns=['insee', 'nom'])
df_com

,insee,nom
0,01001,L'Abergement-Clémenciat
1,01002,L'Abergement-de-Varey
2,01004,Ambérieu-en-Bugey
3,01005,Ambérieux-en-Dombes
4,01006,Ambléon
...,...,...
34447,95676,Villers-en-Arthies
34448,95678,Villiers-Adam
34449,95680,Villiers-le-Bel
34450,95682,Villiers-le-Sec


In [48]:
len(dict_com)

34452

In [49]:
liste_des_voisins = []
for i,c in df.iterrows():
    try:
        vois = c['6_insee_voisins'].split('|')
        for v in vois:
            liste_des_voisins.append((c['6_insee'],v))
    except:
        print(c['6_insee_voisins'])
        pass

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [50]:
len(liste_des_voisins)

205310

In [51]:
df_voisins = pd.DataFrame(liste_des_voisins, columns = ['Com1','Com2'])
df_voisins

,Com1,Com2
0,01001,01412
1,01001,01093
2,01001,01028
3,01001,01146
4,01001,01351
...,...,...
205305,95690,95295
205306,95690,95040
205307,95690,78329
205308,95690,95024


In [52]:
df_voisins.dtypes

Com1    object
Com2    object
dtype: object

In [53]:
G = nx.from_pandas_edgelist(df_voisins, source='Com1', target='Com2')

In [54]:
len(G.nodes)

34454

In [55]:
G.add_nodes_from((d.insee, dict(d)) for n, d in df_com.iterrows())

In [56]:
G.nodes['01001']

{'insee': '01001', 'nom': "L'Abergement-Clémenciat"}

In [57]:
G.nodes

NodeView(('01001', '01412', '01093', '01028', '01146', '01351', '01188', '01002', '01056', '01277', '01384', '01007', '01363', '01199', '01004', '01421', '01041', '01345', '01089', '01149', '01005', '01382', '01207', '01261', '01362', '01318', '01398', '01446', '01006', '01358', '01110', '01117', '01216', '01233', '01190', '01314', '01430', '01304', '01008', '01431', '01202', '01213', '01009', '01098', '01034', '01015', '01116', '01010', '01407', '74269', '73180', '73286', '01138', '01453', '01118', '01011', '01087', '01269', '01265', '01237', '01181', '01031', '01283', '01012', '01191', '01080', '01185', '01155', '01279', '01121', '01013', '01416', '01066', '01280', '01111', '01017', '01014', '39579', '01152', '01148', '01061', '01294', '01310', '01338', '01016', '01402', '01437', '01094', '01337', '71550', '71195', '01019', '01452', '01329', '01311', '01021', '01423', '01250', '01166', '01075', '01022', '01415', '01073', '01372', '01036', '01023', '01057', '01284', '01231', '01439', 

In [58]:
nx.write_gexf(G, f'./Graph/Liste_des_communes_voisines.gexf', encoding='utf-8', prettyprint=True, version='1.2draft')